# AI Alpha Backtest
## Considerations in the backtest
In the backtest, the followings will be need to be additionally considered:
### Time delay in live trading
Realistically, there will be a two days time delay in live trading as PnL is reported for the date of returns.
 - T+0 EOD Factors -> Decide trading
 - T+1 Trade execution on T+1 Price/Market
 - T+2 Calculate returns on EOD Price
The negative impact of the continuous trade executions on the return date [T+2]  based on the trading decisions on T+1 will be factored in to Transaction Cost.

### Remove look ahead bias
 - Universe should be selected on each day to avoid survivership bias.
 - Should use only the data available to each day. Do not calculate the aplha vector by an estimator trained using future date. Continuously training the estimator using the latest data available for each day.

### Trading Cost and Liquidity
Construct the universe of stocks as only those companies
 - that have a large liquidity which has capacity to absorb all trades with less price impact OR
 - that are in the previous¥ day's holdings
Explicit cost such as commissions are far smaller than market impact.
Define the transaction cost, or slippage, as to multiply the price change due to market impact by the amount of dollars traded.

Here, use linear impact model because the slope is not available in public.
Good estimate according the past researches is, each 1% of Average Daily Volume Traded moves the price by 10 basis point.

$$
\mbox{tcost_{i,t}} = \% \Delta \mbox{price}_{i,t} \times \mbox{trade}_{i,t} = (0.1 \times (h_{i,t} - h_{i,t-1}) \div \mbox{ADV}_{i,t}) \times (h_{i,t} - h_{i,t-1})
$$

That is,
$$
\mbox{tcost}_{i,t} = \sum_i^{N} \lambda_{i,t} (h_{i,t} - h_{i,t-1})^2
$$  
where
$$
\lambda_{i,t} = \frac{1}{10\times \mbox{ADV}_{i,t}}
$$

When ADV is missing or zero, consider it to be a small number like 10,000 and highly illiquid.

Then, total transaction costs will be:
$$
\mbox{tcost} = \sum_i^{N} \lambda_{i} (h_{i,t} - h_{i,t-1})^2
$$

### Winsorize
Clip the data to avoid extremely positive or negative values to handle noise causing unusually large positions.

## Data available for backtest
This means, the data available for back test would be around 2 years, 2014 and 2015 because
 - Original data: 5 years from Jan 2011 to Jan 2016
 - Factor data: 4 years from Jan 2012 to Jan 2016
 - Weekly return: The last week cannot be used, so from Jan 2012 to Dec 2015
 - Keep rolling window of 2 years for training AI Alpha, resulting in the range from Jan 2014 to Dec 2015

In addition, covariance matrix should be calculated on each day as the universe changes on daily basis.

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

Looking in links: https://files.pythonhosted.org/


In [2]:
import helper

import cvxpy as cvx
import numpy as np
import pandas as pd
import time
from datetime import datetime, date
import importlib

from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

/Users/yuki/miniconda3/envs/ai-alpha/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Configurations
### Data Bundle

In [3]:
import os
from zipline.data import bundles

os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), 'data', 'zipline')

ingest_func = bundles.csvdir.csvdir_equities(['daily'], helper.EOD_BUNDLE_NAME)
bundles.register(helper.EOD_BUNDLE_NAME, ingest_func)

print('Data Registered')

Data Registered


### Universe
Since the liquidity matters, choose top 500 stocks in terms of average dollar volume traded among highly developed market - US Equity.
Bid-ask spead for these equities are 2-5bp, as oppose to 20-30bp for less liquid stocks. To target 400bp annual return from daily trading, more than 10bp for a single trade is not acceptable.

In [4]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar

universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(helper.EOD_BUNDLE_NAME)
engine = helper.build_pipeline_engine(bundle_data, trading_calendar)
sector = helper.Sector()

### Returns

In [5]:
from zipline.data.data_portal import DataPortal

data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

### Define Dates

In [70]:
# Factor start and end date
start_date = pd.Timestamp('2014-01-15', tz='UTC', freq='C')
end_date = pd.Timestamp('2014-12-31', tz='UTC', freq='C')

start_loc = trading_calendar.closes.index.get_loc(start_date)
end_loc = trading_calendar.closes.index.get_loc(end_date)

backtest_dates = [pd.Timestamp(x.strftime('%Y-%m-%d'), tz='UTC', freq='C') for x in trading_calendar.closes[start_loc+2:end_loc]]
backtest_dates


[Timestamp('2014-01-17 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-21 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-22 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-23 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-24 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-27 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-28 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-29 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-30 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-01-31 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-02-03 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-02-04 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-02-05 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-02-06 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-02-07 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-02-10 00:00:00+0000', tz='UTC', freq='C'),
 Timestamp('2014-02-11 00:00:00+0000', t

### Features for Machine Learning

In [71]:
sector_lookup = pd.read_csv('./data/sector/labels.csv', index_col='Sector_i')['Sector'].to_dict()
sector_columns = []
for sector_i, sector_name in sector_lookup.items():
    secotr_column = 'sector_{}'.format(sector_name)
    sector_columns.append(secotr_column)

In [72]:
features = [
    'Mean_Reversion_Sector_Neutral_Smoothed', 'Momentum_1YR',
    'Overnight_Sentiment_Smoothed', 'adv_120d', 'adv_20d',
    'dispersion_120d', 'dispersion_20d', 'market_vol_120d',
    'market_vol_20d', 'volatility_20d',
    'is_Janaury', 'is_December', 'weekday',
    'month_end', 'month_start', 'qtr_end', 'qtr_start'] + sector_columns
target_label = 'target'

### Hyper Parameters

In [73]:
n_days = 10
n_stocks = 500
clf_random_state = 42

clf_parameters = {
    'n_estimators': 250,
    'criterion': 'entropy',
    'min_samples_leaf': n_stocks * n_days,
    'oob_score': True,
    'n_jobs': -1,
    'random_state': clf_random_state}

## Backtest

In [74]:
def form_optimal_portfolio(factor_date, return_date, previous, universe):
    '''Returns the optimal portfolio, risk and alpha exposures and the total transaction cost'''
    factor_date_loc = trading_calendar.closes.index.get_loc(factor_date)
    factor_start_date = pd.Timestamp(trading_calendar.closes[factor_date_loc-252*3].strftime('%Y-%m-%d'), tz='UTC', freq='C')
    print("Factor Start Date: ", factor_start_date.strftime('%Y%m%d'))
    alpha_factors = helper.get_alpha_factors(universe, sector, engine, factor_start_date, factor_date)
    alpha_factors_today = alpha_factors.loc[alpha_factors.index.get_level_values(0) == factor_date]
    df = alpha_factors_today.reset_index(level=0, drop=True)
    df = df.merge(previous, left_index=True, right_index=True, how='left')
    df = helper.clean_nas(df)

    universe_tickers = helper.get_universe_tickers(factor_date, universe, engine)
    
    three_year_pricing = helper.get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        factor_start_date,
        factor_date)

    three_year_returns = three_year_pricing.pct_change()[1:].fillna(0)
  
    risk_model = helper.get_risk_factors(three_year_returns)
    
    today_pricing = three_year_pricing.loc[three_year_pricing.index.get_level_values(0) == factor_date]

    today_returns = three_year_returns.loc[three_year_returns.index.get_level_values(0) == factor_date]
    today_returns = today_returns.reset_index(level=0, drop=True)

    result = helper.train_model(alpha_factors, features, target_label, clf_parameters)
    alpha_vector = helper.get_alpha_vector(result[0], df, features, target_label)
  
    Lambda = helper.get_lambda(df)

    h_star = helper.OptimalHoldingsStrictFactor(
        weights_max=0.05,
        weights_min=-0.05,
        risk_cap=0.0015,
        factor_max=0.035,
        factor_min=-0.035).find(alpha_vector,
                                risk_model['factor_betas'], 
                                risk_model['factor_cov_matrix'], 
                                risk_model['idiosyncratic_var_vector'])
    h_star = h_star * 1e6 
    
    port_risk = helper.predict_portfolio_risk(
        risk_model['factor_betas'],
        risk_model['factor_cov_matrix'],
        risk_model['idiosyncratic_var_matrix'],
        h_star)
        
    
    risk_exposures = helper.get_risk_exposures(risk_model['factor_betas'], h_star)
    alpha_exposure = helper.get_alpha_exposures(alpha_vector, h_star)
    total_transaction_costs = helper.get_total_transaction_costs(previous, h_star, Lambda)
    
    alpha_vector_with_date = alpha_vector.set_index(pd.Index([factor_date]*alpha_vector.shape[0], name='date'), append=True)
    alpha_vector_with_date = alpha_vector_with_date.swaplevel()
    
    return {
        "opt.portfolio" : h_star,
        "portfolio.risk": port_risk,
        "risk.return" : risk_model['factor_returns'], 
        "risk.exposures" : risk_exposures, 
        "alpha.exposures" : alpha_exposure,
        "total.cost" : total_transaction_costs,
        "alpha.vector": alpha_vector_with_date,
        "today.returns": today_returns}

def build_tradelist(prev_holdings, opt_result):
    '''Returns the trades on the day'''
    tmp = prev_holdings.merge(opt_result['opt.portfolio'], how='outer', left_index=True, right_index=True)
    tmp['h.opt.previous'] = np.nan_to_num(tmp['h.opt.previous'])
    tmp['h.opt'] = np.nan_to_num(tmp['h.opt'])
    return tmp

def convert_to_previous(result): 
    '''Save optimal holdings as previous optimal holdings'''
    prev = result['opt.portfolio']
    prev = prev.rename(index=str, columns={"h.opt": "h.opt.previous"}, copy=True, inplace=False)
    return prev

## Run the backtest

In [ ]:
importlib.reload(helper)
result = {}
trades = {}
portfolio = {}

universe_tickers = helper.get_universe_tickers(backtest_dates[0], universe, engine)
previous_holdings = pd.DataFrame(data = {"h.opt.previous" : np.array(0)}, index=universe_tickers)


for dt in tqdm(backtest_dates, desc='Optimizing Portfolio', unit='day'):
    date = dt.strftime('%Y%m%d')
    return_date = dt
    print("Return Date: ", return_date.strftime('%Y%m%d'))
    return_date_loc = trading_calendar.closes.index.get_loc(return_date)
    factor_date = pd.Timestamp(trading_calendar.closes[return_date_loc-2].strftime('%Y-%m-%d'), tz='UTC', freq='C')
    print("Factor Date", factor_date.strftime('%Y%m%d'))
    
    result[date] = form_optimal_portfolio(factor_date, return_date, previous_holdings, universe)
    trades[date] = build_tradelist(previous_holdings, result[date])
    portfolio[date] = result[date]
    previous_holdings = convert_to_previous(result[date])
    

Optimizing Portfolio:   0%|          | 0/240 [00:00<?, ?day/s]

Return Date:  20140117
Factor Date 20140115
Factor Start Date:  20110112


Optimizing Portfolio:   0%|          | 1/240 [00:44<2:56:02, 44.19s/day]

Return Date:  20140121
Factor Date 20140116
Factor Start Date:  20110113


Optimizing Portfolio:   1%|          | 2/240 [01:24<2:47:04, 42.12s/day]

...

Return Date:  20140203
Factor Date 20140130
Factor Start Date:  20110127


Optimizing Portfolio:   5%|▍         | 11/240 [08:28<2:56:32, 46.25s/day]

Return Date:  20140204
Factor Date 20140131
Factor Start Date:  20110128


Optimizing Portfolio:   5%|▌         | 12/240 [09:22<2:58:03, 46.86s/day]

Return Date:  20140205
Factor Date 20140203
Factor Start Date:  20110131


## PnL Attribution
Profit and Loss is the aggregate realized daily returns of the assets, weighted by the optimal portfolio holdings chosen, and summed up to get the portfolio's profit and loss.

The PnL attributed to the alpha factors equals the factor returns times factor exposures for the alpha factors.  

$$
\mbox{PnL}_{alpha}= f \times b_{alpha}
$$

Similarly, the PnL attributed to the risk factors equals the factor returns times factor exposures of the risk factors.

$$
\mbox{PnL}_{risk} = f \times b_{risk}
$$

In [ ]:
def partial_dot_product(v, w):
    '''Return sum of two Pandas Series for common indices.'''
    common = v.index.intersection(w.index)
    return np.sum(v[common] * w[common])

def build_pnl_attribution():
    new=1
    for dt in backtest_dates:
        date = dt.strftime('%Y%m%d')
        p = portfolio[date]
        if new:
            alpha_vector = p['alpha.vector']
            new = 0
        else:
            alpha_vector = pd.concat([alpha_vector, p['alpha.vector']])
    
    all_assets = alpha_vector.index.levels[1].values.tolist()
    all_pricing = helper.get_pricing(
        data_portal,
        trading_calendar,
        all_assets,
        start_date,
        end_date)
    # print(alpha_vector)
    factor_data = helper.build_factor_data(alpha_vector, all_pricing)
    
    alpha_returns = helper.get_factor_returns(factor_data)
    qr_alpha_returns = helper.get_qr_factor_returns(factor_data)
    sharpe_ratio = helper.sharpe_ratio(alpha_returns)
    
    df = pd.DataFrame(index = backtest_dates)
    
    for dt in tqdm(backtest_dates[:-3]):
        date = dt.strftime('%Y%m%d')

        p = portfolio[date]
        
        alpha_returns_today = alpha_returns.loc[alpha_returns.index.get_level_values(0) == dt]
        alpha_returns_today = alpha_returns_today.reset_index(level=0, drop=True)

        mf = p['opt.portfolio'].merge(p['today.returns'].T, how='left', left_index=True, right_index=True)
        mf['today.returns'] = helper.wins(mf[0], -0.5, 0.5)
        df.at[dt,"daily.pnl"] = np.sum(mf['h.opt'] * mf['today.returns'])
        
        # df.at[dt,"attribution.alpha.pnl"] = partial_dot_product(alpha_returns_today, p['alpha.exposures'])
        df.at[dt,"attribution.alpha.pnl"] = alpha_returns_today.values[0][0] * p['alpha.exposures'].values[0][0]
        #df.at[dt,"attribution.risk.pnl"] = partial_dot_product(p['risk.return'].iloc[-1], p['risk.exposures']) 
        df.at[dt,"attribution.risk.pnl"] = np.dot(np.asarray(p['risk.return'].iloc[-1]), np.asarray(p['risk.exposures']))[0]
        df.at[dt,"attribution.cost"] = p['total.cost'] * -1
        
    return df 

attr = build_pnl_attribution()
attr

In [ ]:
# Plot
plt.figure(figsize=(15,7))
for column in attr.columns:
        plt.plot(attr[column].cumsum(), label=column)
plt.legend(loc='upper left')
plt.xlabel('Date')
plt.ylabel('PnL Attribution')
plt.show()

In [ ]:
def build_portfolio_characteristics(): 
    df = pd.DataFrame(index = backtest_dates)
    
    for dt in backtest_dates:
        date = dt.strftime('%Y%m%d')
  
        p = portfolio[date]
        tradelist = trades[date]
        h = p['opt.portfolio']['h.opt']
        
        df.at[dt,"long"] = np.sum(h[h>0])
        df.at[dt,"short"] = np.sum(h[h<0])
        df.at[dt,"net"] = np.sum(h)
        df.at[dt,"gmv"] = np.sum(np.abs(h))
        df.at[dt,"traded"] = np.sum(abs(tradelist['h.opt'] - tradelist['h.opt.previous']))
        
    return df

pchar = build_portfolio_characteristics()
pchar

In [ ]:
# Plot
plt.figure(figsize=(15,7))
for column in pchar.columns:
        plt.plot(pchar[column], label=column)
plt.legend(loc='upper left')
plt.xlabel('Date')
plt.ylabel('Portfolio')
plt.show()

## Further consideration
### Objective Function
It is not efficient to optimize the weight by imposing constraints. Equality constraints may result in no optima, whereas unequality constraints all tend to end up local optima or partial optimization.
The objective function to minimize can be given as:

$$\begin{align} f(\mathbf{h}) &= \mbox{factor risk (Risk aversion} \times \mbox{Common Risk)} \\
 &+\mbox{idiosyncratic risk (Risk Aversion} \times \mbox{Specific Risk)} \\
 &-\mbox{expected portfolio return} \\
 &+ \mbox{transaction costs} \\
 &= \frac{1}{2}\kappa \mathbf{h}_t^T\mathbf{Q}^T\mathbf{Q}\mathbf{h}_t + \frac{1}{2} \kappa \mathbf{h}_t^T \mathbf{S} \mathbf{h}_t - \mathbf{\alpha}^T \mathbf{h}_t + (\mathbf{h}_{t} - \mathbf{h}_{t-1})^T \mathbf{\Lambda} (\mathbf{h}_{t} - \mathbf{h}_{t-1}) \end{align}
$$

where $\textbf{B}$ is $N$ by $K$ matrix, $\textbf{F}$ is $K$ by $K$ matrix and $\textbf{B}^T$ is $K$ by $N$ matrix, resulting in $\textbf{BFB}^T$ as a huge $N \times N$ matrix (e.g. 2,000 assets $\times$ 2,000 assets = 2 million)
To avoid this, covert it to $\textbf{Q}^T\textbf{Q}$, by the following:
$\textbf{BFB}^T$ = $\textbf{BGGB}^T$ = $\textbf{Q}^T\textbf{Q}$, where $\textbf{Q} = \textbf{GB}^T$. Then the resulting $\textbf{Qh} = \textbf{R}$, and $\textbf{Q}^T\textbf{h}^T = \textbf{R}^T$ are just K by 1 vector and 1 by K vector, respectively.

$\kappa$ is risk aversion factor, applied to any size metrics. (e.g. $GMV = \sum_i^{N}(|\mathbf{h}_i|)$ is a L1-norm)

This model include aspects of Market neutral, Position Size, Portfolio Diversification without using Constraints, which is high in computational cost.
This will be considered as the next step.

### Risk Factors
Rather than PCA, using commertial factor models are more realistic such as MSCI Barra. To be considered as the next step when using more data.